In [3]:
#Berikut ini adalah code untuk menampilkan video live stream dari kamera 
#framenya diperkecil menjadi setengahnya
#untuk menutup frame, klik key 'q'

import cv2

cap = cv2.VideoCapture(0)

# Check if the webcam is opened correctly
if not cap.isOpened():
    raise IOError("Cannot open webcam")
    
while True:
    ret, frame = cap.read()
    frame = cv2.resize(frame, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_AREA)
    cv2.imshow('Input', frame)
    
    if cv2.waitKey(1) & 0xff == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

In [4]:
#Berikut ini merupakan code yang menampilkan video live stream dari kamera
#kemudian anda dapat menggunakan mouse input dan membuat persegi di mana
#gambar pada persegi akan diubah menjadi citra negatif

import cv2
import numpy as np

def draw_rectangle(event, x, y, flags, params):
    global x_init, y_init, drawing, top_left_pt, bottom_right_pt
    
    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        x_init, y_init = x, y
    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing:
            top_left_pt = (min(x_init, x), min(y_init, y))
            bottom_right_pt = (max(x_init, x), max(y_init, y))
            img[y_init:y, x_init:x] = 255 - img[y_init:y, x_init:x] #proses mengubah jadi negatif
    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        top_left_pt = (min(x_init, x), min(y_init, y))
        bottom_right_pt = (max(x_init, x), max(y_init, y))
        img[y_init:y, x_init:x] = 255 - img[y_init:y, x_init:x] #proses mengubah jadi negatif

if __name__=='__main__':
    drawing = False
    top_left_pt, bottom_right_pt = (-1,-1), (-1,-1)
    
    cap = cv2.VideoCapture(0)
    
    # Check if the webcam is opened correctly
    if not cap.isOpened():
        raise IOError("Cannot open webcam")
        
    cv2.namedWindow('Webcam')
    cv2.setMouseCallback('Webcam', draw_rectangle)
    
    while True:
        ret, frame = cap.read()
        img = cv2.resize(frame, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_AREA)
        
        (x0,y0), (x1,y1) = top_left_pt, bottom_right_pt
        img[y0:y1, x0:x1] = 255 - img[y0:y1, x0:x1]
        
        cv2.imshow('Webcam', img)
        
        if cv2.waitKey(1) & 0xff == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
#Tugasnya
#Silakan anda bereksperimen dengan bagian persegi yang dipilih pada video live stream dari camera
#Jika pada contoh bagian persegi diubah menjadi negatif,
#maka pada tugas ini, anda bisa berkreasi, seperti mengubah warna, apply filter seperti blur,
#gaussianblur, medianblur, atau menampilkan efek lainnya

#atau anda bisa berkreasi pada keseluruhan gambar dengan menambah facemask, kumis, 
#telinga kucing atau lainnya

In [5]:
# PENCIL SKETCH EFEK

import cv2 as cv
import numpy as np

camera = cv.VideoCapture(0)

while True:
    ret, frame = camera.read()
    if ret == False:
        break
    # getting width and height of image
    height, width, _ = frame.shape

    # creating copy of image using resize function
    resziedImage = cv.resize(frame, (width, height),
                             interpolation=cv.INTER_AREA)

    # creating 3X3 kernel, inorder to sharpen the image /frame
    kernel = np.array([[-1, -1, -1],
                       [-1, 9, -1],
                       [-1, -1, -1]])
    # Appling kernel on the frame using filter2D function .
    sharpenImage = cv.filter2D(resziedImage, -1, kernel)

    # converting image into Grayscale image.
    gray = cv.cvtColor(sharpenImage, cv.COLOR_BGR2GRAY)

    # creating inverse of sharpen image .
    inverseImage = 255-gray

    # applying Gussain Blur on the image .
    bluredImage = cv.GaussianBlur(inverseImage, (15, 15), 0, 0)

    # create a pencilSketch using divide function on opencv .
    pencilSketch = cv.divide(gray, 255-bluredImage, scale=256)

    # show the frame on the screen .
#     cv.imshow('Sharpen Image', sharpenImage)
    cv.imshow("pencilSketch", pencilSketch)

#     cv.imshow("frame", frame)
    key = cv.waitKey(1)
    if key == ord('q'):
        break

cv.destroyAllWindows()
camera.release()

In [6]:
# filter kaca mata
import cv2

cap = cv2.VideoCapture(0)

cascade = cv2.CascadeClassifier('tugas1/face.xml')

specs_ori = cv2.imread("tugas1/glass.png" , -1)
cigar_ori = cv2.imread("tugas1/cigar.png" , -1)
mus_ori = cv2.imread("tugas1/mustache.png" , -1)

def transparentOverlay(src, overlay , pos = (0,0)  , scale = 1):
    overlay = cv2.resize(overlay , (0,0) ,fx = scale , fy = scale)
    h , w , _ =  overlay.shape ## size of foreground image
    rows , cols , _ = src.shape  ## size of background image
    y , x = pos[0] , pos [1]

    for i in range(h):
        for j in range(w):
            if x + i > rows or y + j >=cols:
                continue
            alpha = float(overlay[i][j][3]/255) ##  read the alpha chanel
            src[x+i][y+j] = alpha * overlay[i][j][:3] + (1-alpha) * src[x+i][y+j]
    return src


while cap.isOpened():
    result , frame = cap.read()
    if result:
        gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        faces =cascade.detectMultiScale(gray ,1.3 , 5 , 0 , minSize=(120,120) , maxSize=(350,350))
        for (x,y,w,h) in faces:
            if h> 0 and w>0:
                glass_symin = int(y+ 1.5 * h/5)
                glass_symax = int(y + 2.5 * h / 5)
                sh_glass = glass_symax - glass_symin

                cigar_symin = int(y + 4 * h / 6)
                cigar_symax = int(y + 5.5 * h / 6)
                sh_cigar = cigar_symax - cigar_symin

                mus_symin = int(y + 3.5 * h / 6)
                mus_symax = int(y + 5 * h / 6)
                sh_mus = mus_symax - mus_symin

                face_glass_ori = frame [glass_symin:glass_symax , x:x+w]
                cigar_glass_ori = frame[cigar_symin:cigar_symax, x:x + w]
                mus_glass_ori = frame[mus_symin:mus_symax, x:x + w]

                glass = cv2.resize(specs_ori , (w , sh_glass) , interpolation= cv2.INTER_CUBIC)
                cigar = cv2.resize(cigar_ori, (w, sh_cigar), interpolation=cv2.INTER_CUBIC)
                mus = cv2.resize(mus_ori, (w, sh_mus), interpolation=cv2.INTER_CUBIC)

                transparentOverlay(face_glass_ori  , glass)
                transparentOverlay(mus_glass_ori , mus)


        cv2.imshow("frame" , frame)
        if cv2.waitKey(1)== ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [7]:
# filter crown
import dlib
import numpy as np
import cv2
from math import hypot

cap = cv2.VideoCapture(0)
headImage = cv2.imread("tugas1/crown.png")

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("tugas1/shape_predictor_68_face_landmarks.dat")

while True:
    _, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    faces = detector(gray)
    
    for face in faces: 
        landmarks = predictor(gray, face)
        #cv2.circle(frame, tophead, 5, (0,0,255), -1)
        lefthead = (landmarks.part(19).x,landmarks.part(19).y)
        righthead = (landmarks.part(24).x,landmarks.part(24).y)
        centerhead = (landmarks.part(27).x,landmarks.part(27).y)
        headWidth = int(hypot(lefthead[0] - righthead[0],
                          lefthead[1] - righthead[1]) *2.5)
        headHeight = headWidth 
        
        topLeft = (int(centerhead[0]-headWidth/2),int(centerhead[1]-(headHeight)))
        #bottomRight = (int(centerhead[0]+headWidth/2),int(centerhead[1]+headHeight/2))
        
        crownImage = cv2.resize(headImage, (headWidth,headHeight))
        crownImageGray = cv2.cvtColor(crownImage, cv2.COLOR_BGR2GRAY)
        _, headMask = cv2.threshold(crownImageGray, 25, 255, cv2.THRESH_BINARY_INV)
        
        headArea = frame[topLeft[1] :  topLeft[1]+headHeight,
                         topLeft[0] :  topLeft[0]+ headWidth]  
        headAreaNohead = cv2.bitwise_and(headArea,headArea, mask = headMask)
        finalhead = cv2.add(headAreaNohead,crownImage)
        
        frame[topLeft[1] : topLeft[1] + headHeight,
                         topLeft[0] : topLeft[0] + headWidth]  = finalhead
                     
        
        
    cv2.imshow("frame", frame)
        
    key = cv2.waitKey(1)
    if key == 27:
        cap.release()
        cv2.destroyAllWindows()
        break
        

In [8]:
# filter love

import dlib
import numpy as np
import cv2
from math import hypot

cap = cv2.VideoCapture(0)
headImage = cv2.imread("tugas1/heart.png")

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("tugas1/shape_predictor_68_face_landmarks.dat") #google for this file and download it from github.

while True:
    _, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    faces = detector(gray)
    
    for face in faces: 
        landmarks = predictor(gray, face)
        #cv2.circle(frame, tophead, 5, (0,0,255), -1)
        lefthead = (landmarks.part(19).x,landmarks.part(19).y)
        righthead = (landmarks.part(24).x,landmarks.part(24).y)
        centerhead = (landmarks.part(27).x,landmarks.part(27).y)
        headWidth = int(hypot(lefthead[0] - righthead[0],
                          lefthead[1] - righthead[1]) *2.5)
        headHeight = headWidth 
        
        topLeft = (int(centerhead[0]-headWidth/2),int(centerhead[1]-(headHeight)))
        #bottomRight = (int(centerhead[0]+headWidth/2),int(centerhead[1]+headHeight/2))
        
        heartImage = cv2.resize(headImage, (headWidth,headHeight))
        heartImageGray = cv2.cvtColor(heartImage, cv2.COLOR_BGR2GRAY)
        _, headMask = cv2.threshold(heartImageGray, 25, 255, cv2.THRESH_BINARY_INV)
        
        headArea = frame[topLeft[1] :  topLeft[1]+headHeight,
                         topLeft[0] :  topLeft[0]+ headWidth]  
        headAreaNohead = cv2.bitwise_and(headArea,headArea, mask = headMask)
        finalhead = cv2.add(headAreaNohead,heartImage)
        
        frame[topLeft[1] : topLeft[1] + headHeight,
                         topLeft[0] : topLeft[0] + headWidth]  = finalhead            
        
        
    cv2.imshow("frame", frame)
        
    key = cv2.waitKey(1)
    if key == 27:
        cap.release()
        cv2.destroyAllWindows()
        break
        

In [9]:
# filter mahkota pink

import dlib
import numpy as np
import cv2
from math import hypot

cap = cv2.VideoCapture(0)
headImage = cv2.imread("tugas1/pink_crown.png")

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("tugas1/shape_predictor_68_face_landmarks.dat") #google for this file and download it from github.

while True:
    _, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    faces = detector(gray)
    
    for face in faces: 
        landmarks = predictor(gray, face)
        #cv2.circle(frame, tophead, 5, (0,0,255), -1)
        lefthead = (landmarks.part(19).x,landmarks.part(19).y)
        righthead = (landmarks.part(24).x,landmarks.part(24).y)
        centerhead = (landmarks.part(27).x,landmarks.part(27).y)
        headWidth = int(hypot(lefthead[0] - righthead[0],
                          lefthead[1] - righthead[1]) * 2)
        headHeight = headWidth 
        
        topLeft = (int(centerhead[0]-headWidth/2),int(centerhead[1]-(headHeight)))
        #bottomRight = (int(centerhead[0]+headWidth/2),int(centerhead[1]+headHeight/2))
        
        crownImage = cv2.resize(headImage, (headWidth,headHeight))
        crownImageGray = cv2.cvtColor(crownImage, cv2.COLOR_BGR2GRAY)
        _, headMask = cv2.threshold(crownImageGray, 25, 255, cv2.THRESH_BINARY_INV)
        
        headArea = frame[topLeft[1] :  topLeft[1]+headHeight,
                         topLeft[0] :  topLeft[0]+ headWidth]  
        headAreaNohead = cv2.bitwise_and(headArea,headArea, mask = headMask)
        finalhead = cv2.add(headAreaNohead,crownImage)
        
        frame[topLeft[1] : topLeft[1] + headHeight,
                         topLeft[0] : topLeft[0] + headWidth]  = finalhead
        
#         cv2.imshow("head area", headArea)
#         cv2.imshow("crown head",crownImage)
#         cv2.imshow("final head",finalhead)             
        
        
    cv2.imshow("frame", frame)
        
    key = cv2.waitKey(1)
    if key == 27:
        cap.release()
        cv2.destroyAllWindows()
        break
        

In [10]:
# filter hidung babi

import cv2
import numpy as np
import dlib
from math import hypot

#start camera
cap = cv2.VideoCapture(0)
nose_image = cv2.imread("tugas1/pig_nose.png")

#load face detector
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("tugas1/shape_predictor_68_face_landmarks.dat")

while True:
    # Capture frame-by-frame
    _,frame = cap.read()
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(frame)
    for face in faces:
        landmarks = predictor(gray_frame,face)
        # Nose coordinates
        top_nose = (landmarks.part(29).x, landmarks.part(29).y)
        center_nose = (landmarks.part(30).x, landmarks.part(30).y)
        #cv2.circle(frame, center_nose, 3, (255,0,0), -1)
        left_nose = (landmarks.part(19).x, landmarks.part(19).y)
        right_nose = (landmarks.part(24).x, landmarks.part(24).y)
        
        nose_width = int(hypot(left_nose[0]-right_nose[0], left_nose[1]-right_nose[1]) * 0.7)
        nose_height = int(nose_width * 0.9)
        
        # New nose position
        top_left = (int(center_nose[0] - nose_width/2), int(center_nose[1] - nose_height/2))
        bottom_right = (int(center_nose[0] + nose_width/2), int(center_nose[1] + nose_height/2))
        #cv2.rectangle(frame, (int(center_nose[0] - nose_width/2), int(center_nose[1] - nose_height/2)),
        #             (int(center_nose[0] + nose_width/2), int(center_nose[1] + nose_height/2)), (0,255,0), 2)
        
        nose_pig = cv2.resize(nose_image, (nose_width, nose_height))
        nose_pig_gray = cv2.cvtColor(nose_pig, cv2.COLOR_BGR2GRAY)
        
        _,nose_mask = cv2.threshold(nose_pig_gray, 25, 255, cv2.THRESH_BINARY_INV)
        
        nose_area = frame[top_left[1]:top_left[1]+nose_height, top_left[0]:top_left[0]+nose_width]
        
        nose_area_no_nose = cv2.bitwise_and(nose_area, nose_area, mask=nose_mask)
        
        final_nose = cv2.add(nose_area_no_nose, nose_pig)
        frame[top_left[1]:top_left[1]+nose_height, top_left[0]:top_left[0]+nose_width] = final_nose
        
        
    cv2.imshow("Frame",frame)
    
    key = cv2.waitKey(1)
    if key == 27:
        cap.release()
        cv2.destroyAllWindows()
        break

In [11]:
# filter horn halo, marah muncul tanduk, senyum muncul mahkota kuning

import dlib
import numpy as np
import cv2
from math import hypot

cap = cv2.VideoCapture(0)
img1 = cv2.imread("tugas1/leftHorn.png")
img2 = cv2.imread("tugas1/rightHorn.png")
img3 = cv2.imread("tugas1/new.png")

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("tugas1/shape_predictor_68_face_landmarks.dat") 

while True:
    _, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) #convert the video into gray for faster processing
    
    faces = detector(gray) #detect faces in the gray frame 
    for face in faces:
        x1 = face.left()
        y1 = face.top()
        x2 = face.right()
        y2 = face.bottom()
        faceWidth = x2 - x1
        faceHeight = y2 - y1
        landmarks = predictor(gray, face)
        leftEyebrow = (landmarks.part(21).x,landmarks.part(21).y)
        rightEyebrow = (landmarks.part(22).x,landmarks.part(22).y)
        leftHeadLeft = (landmarks.part(0).x,landmarks.part(0).y)
        leftHeadRight = (landmarks.part(20).x,landmarks.part(20).y)
        leftLip = (landmarks.part(48).x,landmarks.part(48).y)
        rightLip = (landmarks.part(54).x,landmarks.part(54).y)
        
        hornWidth = int(hypot(leftHeadLeft[0] - leftHeadRight[0],
                              leftHeadLeft[1] - leftHeadRight[1]))
        hornHeight = int(1.2 * hornWidth)

        haloWidth = int((x2 - x1))
        haloHeight = int(haloWidth * 0.4)
        
        if (rightEyebrow[0] - leftEyebrow[0])/faceWidth < 0.13:
#             print("angry")
            
            #left horn : 
            leftHornCenter = (landmarks.part(18).x ,landmarks.part(18).y - 20)
            leftTopLeft = (int(leftHornCenter[0] - hornWidth / 2),int(leftHornCenter[1] - hornHeight / 2))
            leftBottomRight = (int(leftHornCenter[0] + hornWidth / 2),int(leftHornCenter[1] + hornHeight / 2))
            
            leftHornImage = cv2.resize(img1, (hornWidth,hornHeight))
            leftHornImageGray = cv2.cvtColor(leftHornImage, cv2.COLOR_BGR2GRAY)
            _, leftHornMask = cv2.threshold(leftHornImageGray, 25, 255, cv2.THRESH_BINARY_INV)
            
            leftHornArea = frame[leftTopLeft[1] : leftTopLeft[1] + hornHeight,
                             leftTopLeft[0] : leftTopLeft[0] + hornWidth]
            
            leftHornAreaNoHorn = cv2.bitwise_and(leftHornArea,leftHornArea, mask = leftHornMask)
            
            finalLeftHorn = cv2.add(leftHornAreaNoHorn,leftHornImage)
            frame[leftTopLeft[1] : leftTopLeft[1] + hornHeight,leftTopLeft[0] : leftTopLeft[0] + hornWidth] = finalLeftHorn
            
            #right horn : 
            
            rightHornCenter = (landmarks.part(25).x ,landmarks.part(25).y - 20)
            rightTopLeft = (int(rightHornCenter[0] - hornWidth / 2),int(rightHornCenter[1] - hornHeight / 2))
            rightBottomRight = (int(rightHornCenter[0] + hornWidth / 2),int(rightHornCenter[1] + hornHeight / 2))
            
            rightHornImage = cv2.resize(img2, (hornWidth,hornHeight))
            rightHornImageGray = cv2.cvtColor(rightHornImage, cv2.COLOR_BGR2GRAY)
            _, rightHornMask = cv2.threshold(rightHornImageGray, 25, 255, cv2.THRESH_BINARY_INV)
            
            rightHornArea = frame[rightTopLeft[1] : rightTopLeft[1] + hornHeight,
                             rightTopLeft[0] : rightTopLeft[0] + hornWidth]
            
            rightHornAreaNoHorn = cv2.bitwise_and(rightHornArea,rightHornArea, mask = rightHornMask)
            
            finalRightHorn = cv2.add(rightHornAreaNoHorn,rightHornImage)
            frame[rightTopLeft[1] : rightTopLeft[1] + hornHeight,rightTopLeft[0] : rightTopLeft[0] + hornWidth] = finalRightHorn
            
            
        elif ((rightLip[0] - leftLip[0])/faceWidth) > 0.43:
#             print("smiling")
            haloCenter = (int((x1 + x2) / 2), y1 - 10)
            haloTopLeft = (int(haloCenter[0] - haloWidth/2),int(haloCenter[1] - haloHeight/2))
            haloBottomRight = (int(haloCenter[0] + haloWidth/2),int(haloCenter[1] + haloHeight/2))
            
            haloImage = cv2.resize(img3, (haloWidth,haloHeight))
            haloImageGray = cv2.cvtColor(haloImage, cv2.COLOR_BGR2GRAY)
            _, haloMask = cv2.threshold(haloImageGray, 25, 255, cv2.THRESH_BINARY_INV) 
            
            haloArea = frame[haloTopLeft[1] : haloTopLeft[1] + haloHeight,
                             haloTopLeft[0] : haloTopLeft[0] + haloWidth]
            
            haloAreaNoHalo = cv2.bitwise_and(haloArea,haloArea, mask = haloMask)
            
            finalHalo = cv2.add(haloAreaNoHalo,haloImage)
            
            frame[haloTopLeft[1] : haloTopLeft[1] + haloHeight,haloTopLeft[0] : haloTopLeft[0] + haloWidth] = finalHalo
            
    
    cv2.imshow("frame", frame)
    
    key = cv2.waitKey(1)
    if key == 27:
        cap.release()
        cv2.destroyAllWindows()
        break

In [12]:
# saat sudah di running, akan tampil satu window hasil capt kamera. selanjutnya tarik satu pola segiempat dengan mouse
# lalu klik 'esc' maka akan terbentuk satu petak sktech seperti arsiran pesil. jika ingin menutup window klik 'q'


import cv2
from matplotlib import pyplot as plt
import numpy as np


def sketch_transform(image):
    image_grayscale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image_grayscale_blurred = cv2.GaussianBlur(image_grayscale, (7,7), 0)
    image_canny = cv2.Canny(image_grayscale_blurred, 10, 80)
    _, mask = image_canny_inverted = cv2.threshold(image_canny, 30, 255, cv2.THRESH_BINARY_INV)
    return mask


cam_capture = cv2.VideoCapture(0)
cv2.destroyAllWindows()

while True:
    _, im0 = cam_capture.read()
    showCrosshair = False
    fromCenter = False
    r = cv2.selectROI("Image", im0, fromCenter, showCrosshair)
    break
    
while True:   
    _, image_frame = cam_capture.read()
    
    rect_img = image_frame[int(r[1]):int(r[1]+r[3]), int(r[0]):int(r[0]+r[2])]
    
    sketcher_rect = rect_img
    sketcher_rect = sketch_transform(sketcher_rect)
    
    #Conversion for 3 channels to put back on original image (streaming)
    sketcher_rect_rgb = cv2.cvtColor(sketcher_rect, cv2.COLOR_GRAY2RGB)
    
    #Replacing the sketched image on Region of Interest
    image_frame[int(r[1]):int(r[1]+r[3]), int(r[0]):int(r[0]+r[2])] = sketcher_rect_rgb
    
    cv2.imshow("Sketcher ROI", image_frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cam_capture.release()
        cv2.destroyAllWindows()
        break
        
